In [1]:
import pandas as pd
import numpy as np
import yfinance
import datetime

from optionalyzer.blackscholes import BlackScholes
from optionalyzer.options import Call, Put
from optionalyzer.chart import PayoffChart

import plotly.io as pio
pio.renderers.default = "notebook_connected"
pio.templates.default = "plotly_dark"

# `BlackScholes`

This class implements the Black-Scholes model for pricing European options. The class has methods to calculate option prices and greeks. It can also be used to calculate implied volatility given the market price of an option. Initialization of the class does not require any parameter.

In [2]:
bs = BlackScholes()

## Downloading the Optiona Chain

We'll be using `yfinance` to download the option chain of a stock. We'll be using the `AAPL` stock for this example.


In [5]:
apple = yfinance.Ticker("AAPL")

#Load at the money options
expiry_date = "2024-03-08"
options = apple.option_chain(date=expiry_date)
calls = options.calls
puts = options.puts

spot_price = apple.history(period="1d")['Close'][0]
print(spot_price)

#At the money options
calls = calls[((calls['strike'] - spot_price) < 20) & ((calls['strike'] - spot_price) > -20)]
puts = puts[((puts['strike'] - spot_price) < 20) & ((puts['strike'] - spot_price) > -20)]


189.41000366210938


/tmp/ipykernel_29416/1692216881.py:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [6]:
calls = calls[["lastTradeDate", "strike", "lastPrice", "percentChange"]]
puts = puts[["lastTradeDate", "strike", "lastPrice", "percentChange"]]

In [7]:
display(calls)
display(puts)

,lastTradeDate,strike,lastPrice,percentChange
5,2024-02-07 18:48:33+00:00,170.0,19.99,0.0
6,2024-02-07 17:54:20+00:00,175.0,15.14,0.0
7,2024-02-07 20:55:30+00:00,180.0,10.85,0.0
8,2024-02-07 20:59:46+00:00,185.0,7.10,0.0
9,2024-02-07 20:59:09+00:00,190.0,3.95,0.0
10,2024-02-07 20:59:25+00:00,195.0,1.90,0.0
11,2024-02-07 20:56:19+00:00,200.0,0.81,0.0
12,2024-02-07 20:54:09+00:00,205.0,0.37,0.0


,lastTradeDate,strike,lastPrice,percentChange
11,2024-02-07 20:45:40+00:00,170.0,0.28,0.0
12,2024-02-07 20:59:34+00:00,175.0,0.47,0.0
13,2024-02-07 20:55:04+00:00,180.0,0.96,0.0
14,2024-02-07 20:59:09+00:00,185.0,2.03,0.0
15,2024-02-07 20:59:49+00:00,190.0,3.94,0.0
16,2024-02-07 20:59:25+00:00,195.0,7.00,0.0
17,2024-02-07 18:49:32+00:00,200.0,11.31,0.0
18,2024-02-07 16:41:34+00:00,205.0,16.23,0.0


In [8]:
strikes = calls['strike'].values
call_prices = calls['lastPrice'].values
put_prices = puts['lastPrice'].values

In [9]:
today = datetime.datetime.today().strftime("%Y-%m-%d")
today

'2024-02-08'

## Calculating Implied Volatility

First, we'll calculate the implied volatility of the option chain. We'll be using the `implied_volatility` method of the `BlackScholes` class.

In [10]:
id_ = 1
S = spot_price
K = strikes[id_]
r = 0.0375
tau = (
    datetime.datetime.strptime(expiry_date, "%Y-%m-%d")
    - datetime.datetime.strptime(today, "%Y-%m-%d")
).days / 365
option_type = "call"
price = call_prices[id_]

iv_call = bs.implied_volatility(
    option_price=price,
    S=S,
    K=K,
    r=r,
    tau=tau,
    option_type=option_type,
    verbose=1,
)
print(f"Implied Volatility: {iv_call:.2%}")

Optimized Successfully!
Implied Volatility: 18.07%


Let's calculate the implied volatility for a Put option too.

In [11]:
id_ = 1
S = spot_price
K = strikes[id_]
r = 0.0375
tau = (
    datetime.datetime.strptime(expiry_date, "%Y-%m-%d")
    - datetime.datetime.strptime(today, "%Y-%m-%d")
).days / 365
option_type = "put"
price = put_prices[id_]

iv_put = bs.implied_volatility(
    option_price=price,
    S=S,
    K=K,
    r=r,
    tau=tau,
    option_type=option_type,
    verbose=1,
)
print(f"Implied Volatility: {iv_put:.2%}")

Optimized Successfully!
Implied Volatility: 21.79%


# `Option`

Now that we know the implied volatility of the option chain, we can calculate the option prices and greeks. We'll use implied volatility of 0.35.

## Calls

### Call(110)

In [12]:
iv = 0.35

call = Call(strike_price=110, expiry_date=expiry_date)
call

Call(110, 2024-03-08)

In [13]:
price, greeks = call.calculate_price(
    spot_price=spot_price,
    risk_free_rate=r,
    volatility=iv,
    day=today,
    return_greeks=True,
)

In [14]:
print(f"Price: {price:.2f}")
print(f"Greeks: {greeks}")

Price: 79.74
Greeks: {'delta': 0.9999999885106367, 'gamma': 3.5396731591770927e-09, 'theta': -4.112735746331184, 'vega': 3.531361398530469e-06, 'rho': 8.713725010257173}


### Call(130)

In [15]:
iv = 0.35

call = Call(strike_price=130, expiry_date=expiry_date)
call

Call(130, 2024-03-08)

In [16]:
price, greeks = call.calculate_price(
    spot_price=spot_price,
    risk_free_rate=r,
    volatility=iv,
    day=today,
    return_greeks=True,
)

In [17]:
print(f"Price: {price:.2f}")
print(f"Greeks: {greeks}")

Price: 59.80
Greeks: {'delta': 0.9999508243528713, 'gamma': 1.0856175142586502e-05, 'theta': -4.883994888303761, 'vega': 0.010830682978404884, 'rho': 10.29728159895424}


## Puts

### Put(110)

In [18]:
iv = 0.35

put = Put(strike_price=110, expiry_date=expiry_date)
put

Put(110, 2024-03-08)

In [19]:
price, greeks = put.calculate_price(
    spot_price=spot_price,
    risk_free_rate=r,
    volatility=iv,
    day=today,
    return_greeks=True,
)

In [20]:
print(f"Price: {price:.2f}")
print(f"Greeks: {greeks}")

Price: 0.00
Greeks: {'delta': -1.148936334960382e-08, 'gamma': 3.5396731591770927e-09, 'theta': -7.69513735778787e-06, 'vega': 3.531361398530469e-06, 'rho': -1.758338568236223e-07}


### Put(130)

In [21]:
iv = 0.35

put = Put(strike_price=130, expiry_date=expiry_date)
put

Put(130, 2024-03-08)

In [22]:
price, greeks = put.calculate_price(
    spot_price=spot_price,
    risk_free_rate=r,
    volatility=iv,
    day=today,
    return_greeks=True,
)

In [23]:
print(f"Price: {price:.2f}")
print(f"Greeks: {greeks}")

Price: 0.00
Greeks: {'delta': -4.9175647128696376e-05, 'gamma': 1.0856175142586502e-05, 'theta': -0.023498100529237773, 'vega': 0.010830682978404884, 'rho': -0.0007572573351597756}


# `PayoffChart`

## Working with `PayoffChart`

In [24]:
K = 120
expiry_date = "2024-03-21"
expiry_date2 = "2024-04-21"
c = Call(K, expiry_date)
p = Put(K, expiry_date)
c2 = Call(K+10, expiry_date2)
p2 = Put(K+10, expiry_date2)
iv = 0.35
c, p, c2, p2

(Call(120, 2024-03-21),
 Put(120, 2024-03-21),
 Call(130, 2024-04-21),
 Put(130, 2024-04-21))

In [25]:
today = datetime.datetime.today().strftime("%Y-%m-%d")
today

'2024-02-08'

In [26]:
positions = ["short", "short", "long", "long"]
sigmas = [iv]*4
options = [c, p, c2, p2]
spot_price = 120

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

In [27]:
fig = pc.payoff_chart(date=today, range=0.25)

In [28]:
fig = pc.payoff_chart(date=expiry_date, range=0.25)

## Some Strategies

In [29]:
prices = np.arange(100, 150, 5)
options_dict = {}
for price in prices:
    options_dict[f"c_{price}"] = Call(price, expiry_date)
    options_dict[f"p_{price}"] = Put(price, expiry_date)

options_dict

{'c_100': Call(100, 2024-03-21),
 'p_100': Put(100, 2024-03-21),
 'c_105': Call(105, 2024-03-21),
 'p_105': Put(105, 2024-03-21),
 'c_110': Call(110, 2024-03-21),
 'p_110': Put(110, 2024-03-21),
 'c_115': Call(115, 2024-03-21),
 'p_115': Put(115, 2024-03-21),
 'c_120': Call(120, 2024-03-21),
 'p_120': Put(120, 2024-03-21),
 'c_125': Call(125, 2024-03-21),
 'p_125': Put(125, 2024-03-21),
 'c_130': Call(130, 2024-03-21),
 'p_130': Put(130, 2024-03-21),
 'c_135': Call(135, 2024-03-21),
 'p_135': Put(135, 2024-03-21),
 'c_140': Call(140, 2024-03-21),
 'p_140': Put(140, 2024-03-21),
 'c_145': Call(145, 2024-03-21),
 'p_145': Put(145, 2024-03-21)}

In [30]:
mid_date = "2024-02-21"

### Long Call

Long call is a strategy where you buy a call option.

You use a long call if you think the stock price will increase.

In [31]:
options = [options_dict["c_130"]]
positions = ["long"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

### Short Call

In a short call strategy, you sell a call option. This is a bearish strategy.

In [32]:
options = [options_dict["c_130"]]
positions = ["short"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

### Long Put

This is a bearish strategy where you buy a put option.

In [33]:
options = [options_dict["p_130"]]
positions = ["long"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

### Short Put

This is a bullish strategy where you sell a put option.

In [34]:
options = [options_dict["p_130"]]
positions = ["short"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

### Straddle

An options straddle involves buying (or selling) both a call and a put with the same strike price and expiration on the same underlying asset.

#### Long Straddle

A long straddle is specially designed to assist a trader to catch profits no matter where the market decides to go. By purchasing a put and a call, the trader is able to catch the market's move regardless of its direction. If the market moves up, the call is there; if the market moves down, the put is there.

In [35]:
options = [options_dict["c_130"], options_dict["p_130"]]
positions = ["long", "long"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

#### Short Straddle

A short straddle is a strategy that involves selling both a call and a put with the same strike price and expiration on the same underlying asset. The strategy is designed to profit from a decrease in volatility.

In [36]:
options = [options_dict["c_130"], options_dict["p_130"]]
positions = ["short", "short"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

### Strangle

A strangle is an options strategy in which the investor holds a position in both a call and a put option with different strike prices, but with the same expiration date and underlying asset. A strangle is a good strategy if you think the underlying security will experience a large price movement in the near future but are unsure of the direction. However, it is profitable mainly if the asset does swing sharply in price.

This is similar to a straddle, except that the strike prices of the call and put are different.

#### Long Strangle

In [37]:
options = [options_dict["c_140"], options_dict["p_120"]]
positions = ["long", "long"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

#### Short Strangle

In [38]:
options = [options_dict["c_140"], options_dict["p_120"]]
positions = ["short", "short"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

### Butterfly Spread

This is a neutral strategy that uses four options contracts with the same expiration but three different strike prices:

- A higher strike price
- An at-the-money strike price
- A lower strike price


There are variuos types of butterfly spreads. We'll be using the Long Butterfly Spread.

#### Long Butterfly Spread

The long butterfly call spread is created by buying one in-the-money call option with a low strike price, writing two at-the-money call options, and buying one out-of-the-money call option with a higher strike price. The maximum profit is achieved if the price of the underlying at expiration is the same as the written calls. The max profit is equal to the strike of the written option, less the strike of the lower call, premiums, and commissions paid. The maximum loss is the initial cost of the premiums paid, plus commissions.

In [39]:
#Create a long butterfly spread
options = [options_dict["c_115"], options_dict["c_130"], options_dict["c_130"], options_dict["c_145"]]
positions = ["long", "short", "short", "long"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

### Iron Condor

An iron condor is an options strategy consisting of two puts (one long and one short) and two calls (one long and one short), and four strike prices, all with the same expiration date. The iron condor earns the maximum profit when the underlying asset closes between the middle strike prices at expiration. In other words, the goal is to profit from low volatility in the underlying asset.

The construction of the strategy is as follows:

1. Buy one out of the money (OTM) put with a strike price below the current price of the underlying asset. This OTM put option will protect against a significant downside move to the underlying asset. 
2. Sell one OTM or at the money (ATM) put with a strike price closer to the current price of the underlying asset.
3. Sell one OTM or ATM call with a strike price above the current price of the underlying asset.
4. Buy one OTM call with a strike price further above the current price of the underlying asset. This OTM call option will protect against a substantial upside move.


In [40]:
spot_price

120

In [41]:
pe1 = Put(115, expiry_date)
pe2 = Put(125, expiry_date)
ce1 = Call(135, expiry_date)
ce2 = Call(145, expiry_date)

positions = ["long", "short", "short", "long"]
options = [pe1, pe2, ce1, ce2]

In [42]:
sigmas = [iv]*4

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

pc

PayoffChart([Put(115, 2024-03-21), Put(125, 2024-03-21), Call(135, 2024-03-21), Call(145, 2024-03-21)], ['long', 'short', 'short', 'long'], 120)

In [43]:
fig = pc.payoff_chart(date=expiry_date, range=0.25)